# 02: Creating Modules

This section is a summary of https://go.dev/doc/tutorial/create-module

## Objectives

+ Create a module
+ Learn to return and handle errors
+ Handle data in *slices* using Go's dynamically-sized arrays
+ Store key/value pairs in a map
+ Learn the basics of Go's built-in unit testing features
+ Learn how to compile and install your application


## Summary

In this tutorial you'll create two modules. The first one will be a library with is intended to be imported by other libraries or applications; the second is a *caller* application which will use the first.

## Create a module that others can use

In Go, a module is an artifact that collects one or more related packages for a discrete and useful set of functions (e.g., an statistics module with packages for doing statistic analysis so that other statistics applications can use).

Go code is grouped into packages and packages are grouped into modules. A module spcifies the dependencies needed to run your code, and the Go version that should be used.

Modules can be versioned, so that consumer apps calling the functions in your modules can import the module's updated packages and test with the new version.

Let's create a `greetings` module.

You should start by creating a directory for the module and initializing it. We will name it `example.com/greetings`.

Then type the following code:

```go
package greetings

import "fmt"

// Hello returns a greeting for the named person.
func Hello(name string) string {
	message := fmt.Sprintf("Hello to %v", name)
	return message
}
```

Note that we're declaring a function takes a `name` parameter whose type is string, and also returns a string.

| NOTE: |
| :---- |
| In Go, functions whose name starts with a capital letter can be called by a function not in the same package.<br>In Go this is known as an *exported name*. |

Then, we are declaring a variable to hold the greeting. In Go, the `:=` operator is a shortcut for declaring and initializing a variable in one line. You could have also written:

```go
var message string

message = fmt.Sprintf(...)
```

Then, we use the `Sprintf()` function from the `fmt` package to create a string that contains the string we receive as an argument. The `%v` substitutes the parameter's value in the string.

Finally, we return the formatted greeting.

## Calling your code from another module

Now we will create anothe Go module named `example.com/hello`. It has to be created so that `greetings` and `hello` are at the same level.

Then type the following program:

```go
package main

import (
	"fmt"

	"example.com/greetings"
)

func main() {
	// Get a greeting message and print it
	message := greetings.Hello("Jason Isaacs")
	fmt.Println(message)
}
```

Note that we're declaring a main package.

| NOTE: |
| :---- |
| In Go, code executed as an application must be in a `main` package. |

Then, we import two packages the `fmt` package, and `example.com/greeting`. The `import` statements gives our code access to the exported functions from those modules.

We then declare a `main()` function which includes the invocation of the `greetings.Hello()` function from our `greetings` module.

For production use, we'd publish the `example.com/greetings` module in a place where Go could find it to download it.

As we're not there yet, we will need to use the `go mod edit` command to edit the module to redirect the Go tools from its module path `example.com/greetings`(where the module isn't) to the directory in which it is located.

```bash
go mode edit --replace example.com/greetings=../greetings
```

This information will be included as part of the module information in your `go.mod`.

Right after that you can do:

```go
go mod tidy
```

This will synchronize the `example.com/hello` module's dependencies, adding those required by the code (in the `import`) but not yet tracked.

After that, some information will be added to the `go.mod` file with a pseudo-version number used instead of the semantic version number which our greetings module doesn't have yet.

Now, you're ready to run the program that consumes an external module by simply running the following command in the `hello/` directory:

```go
go run .
```

| EXAMPLE: |
| :------- |
| See [01_greetings](01_greetings) for a runnable example. |

## Return and handle an error

In this section, you'll lear how to return an error from the greetings module, and then how to handle it in the caller.


We will start by modifying the code of the `greetings` module to send an error if the `name` argument is empty.

The following code illustrates how to achieve it:

```go
package greetings

import (
	"errors"
	"fmt"
)

// Hello returns a greeting for the named person.
func Hello(name string) (string, error) {
	if name == "" {
		return "", errors.New("empty name")
	}

	message := fmt.Sprintf("Hello to %v", name)
	return message, nil
}
```

Note that:
+ the function signature has change to state that you will be returning two values: a string and an error.<br>The caller will have to check the second value to see if an error occurred.
+ We are using an `if` statement to see if the passed value is empty, and if so, we are returning the tuple `"", error".
+ The `errors.New()` function lets you return an error with an specific message.
+ When there's no error, the function should also return a couple. The `nil` value is used to signify that there's been no error.


Now we can proceed to modify the caller program, which should be aware of the couple of parameters the `Hello()` function is now returning:

```go
package main

import (
	"fmt"
	"log"

	"example.com/greetings"
)

func main() {
	// Set the properties of the predefined Logger
	log.SetPrefix("greetings: ")
	log.SetFlags(0);		// disable time, source file and line no

	// Request a greeting message and print it
	message, err := greetings.Hello("")

	// If an error is returned we call log.Fatal which will print
	// the error and exit the program
	if err != nil {
		log.Fatal(err)
	}

	// Print the message if no error was returned
	fmt.Println(message)
}
```

In the code above:
+ We've configured the `log` package which is the built-in logger for Go. We have configured it with a custom prefix for the log lines, and disabled all the extra info, so that only the messages we pass to the logger are displayed.

+ We change line in which we invoke `greetings.Hello()` so that we get both the message and the error.

+ We include a guard, so if an error is found, we call `log.Fatal()` which will print a message and stop the program.

You've seen the common way to communicate errors in Go: you return errors as values, so that callers can check for them.

| EXAMPLE: |
| :------- |
| See [02_greetings_err_handling](02_greetings_err_handling) for a runnable example. |

## Handling data in slices using Go's dynamic arrays

This section illustrates how you can change your program so that instead of returning always the same message, you return one of several predefined greeting message randomly selected.

To do this you need to use a Go slice. A slice is like an array, except that its size changes dynamically as you add and remove items.

We'll start by adding a slice containing three greeting messages, and then we'll make our code return one of the messages randomly.

```go
package greetings

import (
	"errors"
	"fmt"
	"math/rand"
	"time"
)

// Hello returns a greeting for the named person.
func Hello(name string) (string, error) {
	if name == "" {
		return "", errors.New("empty name")
	}

	message := fmt.Sprintf(randomFormat(), name)
	return message, nil
}


// init sets initial values for variables used in the function
func init() {
	rand.Seed(time.Now().UnixNano())
}

// randomFormat returns one of a set of predefined greeting messages. The
// returned message is selected randomly
func randomFormat() string {
	// A slide of message formats
	formats := []string{
		"Hi, %v. Welcome!",
		"Hello to %v!",
		"Great to see you %v",
	}

	// Return randomly selected message
	return formats[rand.Intn(len(formats))]
}
```

In the code:

+ we define a `randomFormat()` function in charge of returns a randomly selected format string for our message, which we will inject into what we returned from the `Hello()` function.

    Note that `randomFormat()` starts with a lowercase letter, making it accessible only to code in its own package (i.e., it's not exported).

+ Within the function we define a Go slice, which looks like an array. Note that the size of the array is not given when declaring it, but we do specify the type of its elements.

+ We use `rand.Intn()` to get a random integer in the range [0, len(formats)]. `len()` returns the size of the slice.

+ We define an `init()` function. Go executed `init()` functions automatically at program startup after global variables have been initialized. That way, we don't need to set up a mechanism to control when initialize the random seed and how to do it only once.

There's no change on the caller, side, as this was just an internal implementation change that didn't break the API contract.

| EXAMPLE: |
| :------- |
| See [03_greeting_slices](./03_greetings_slices/) for a runnable example. |

## Storing key-value pairs in a map

In this section, we'll add support for getting greetings for multiple people in one request.

This will let us introduce the handling of key-value pairs in maps.

In order to support multiple people in one request, we need handle multiple-value input, and then pair those input with multiple-value output.

Changing the `Hello()` function to support that is not a good idea, as this would break the contract with existing consumers that are expecting the signature:

```go
func Hello(name string) string
```

As a result, it's a better idea to define a new function `Hellos()` to handle this new capability, as adding a new function will not break any existing code:

```go
func Hellos(names []string) (map[string]string, error) {
	messages := make(map[string]string)

	for _, name := range names {
		message, err := Hello(name)
		if err != nil {
			return nil, err
		}

		messages[name] = message
	}

	return messages, nil
}
```

In this code you:

+ Add a `Hellos()` function that receives a slice of names rather than a single name. The output parameter is a tuple whose first element is a map whose keys are strings and values are strings too.

+ Initialized the internal map of messages using `make(map[key-type]value-type)`.

+ Loop through the names your function has received using a for loop. In order to iterate, we've used `range` which returns two value while iterating over the slice: the index (which we don't need, and therefore use `_` while unpacking it), and the value.

+ Associate each of the greetings to the corresponding item in the map.


Now, we can the caller program so that we use `Hellos()` and receive greetings for a bunch of names:

```go
func main() {
	// Set the properties of the predefined Logger
	log.SetPrefix("greetings: ")
	log.SetFlags(0);		// disable time, source file and line no


	// Declare a slice of names
	names := []string{"Jason Isaacs", "Florence Pugh", "Idris Elba"}

	// Request greetings for those names
	messages, err := greetings.Hellos(names)

	// If an error is returned we call log.Fatal which will print
	// the error and exit the program
	if err != nil {
		log.Fatal(err)
	}

	// Print the message if no error was returned
	fmt.Println(messages)
}
```

The only interesting part is the inline declaration of the slice where we assign three names to the slice.

| EXAMPLE: |
| :------- |
| See [04_greetings_maps](04_greetings_maps/) for a runnable example. |

## Adding a test

This section introduces the Go's built-in unit testing capabilities.

You will be adding a unit test for your `Hello()` function.

Testing in Go sits on three important pillars:
+ naming conventions
+ the `"testing"` package
+ the `go test` command

Let's start by creating a file called `greetings_test.go`. Ending a file's name with `_test.go` tells the `go test` command that this file contains test functions.

The type the following code:

```go
package greetings

import (
	"regexp"
	"testing"
)

// TestHelloName calls greetings.Hello with a name checking for a valid return code
func TestHelloName(t *testing.T) {
	name := "Jason Isaacs"
	want := regexp.MustCompile(`\b`+name+`\b`)
	msg, err := Hello("Jason Isaacs")
	if !want.MatchString(msg) || err != nil {
		t.Fatalf(`Hello("Jason Isaacs") = %q, %v, want match for %#q, nil`, msg, err, want)
	}

}

// TestHelloEmpty calls greetings.Hello with an empty string checking for an error
func TestHelloEmpty(t *testing.T) {
	msg, err := Hello("")
	if msg != "" || err == nil {
		t.Fatalf(`Hello("") = %q, %v, want "", error`, msg, err)
	}
}
```

In the code you:

+ Implement the test functions in the same package as the code you're testing.

+ Create two functions for the test use cases: invoking with a name, invoking with an empty name.

+ The names of the functions are `TestXYZ()` where XYZ says something about the test. The signature for those functions is `*testing.T` which is nothing but a pointer to the testing package's `testing.T` type. You use this parameter for reporting and logging from your test.

  + `TestHelloName()` calls the `Hello()` function passing a name value. If the call returns an error, or a string that does not contain the passed name you invoke `t.FatalF()` to notify the error.

  + `TestHelloEmpty()` calls the `Hello()` function with an empty string. If the call doesn't return an error or returns something other that an empty string as the message, you invoke `t.FatalF()` to signal the error.


| NOTE: |
| :---- |
| The `%q` format specifier returns a quoted string. |


In order to run the test, you can run:

```go
go test
```

which will return a succinct:

```
PASS
ok      example.com/greetings   0.001s
```

If you want more detailed info, you can do:
```bash
go test -v
=== RUN   TestHelloName
--- PASS: TestHelloName (0.00s)
=== RUN   TestHelloEmpty
--- PASS: TestHelloEmpty (0.00s)
PASS
ok      example.com/greetings   0.001s
```

You can also check how the failing tests are reported by changing a little bit the Hello implementation so that it returns the message without substituting the variable:

```bash
$ go test
--- FAIL: TestHelloName (0.00s)
    greetings_test.go:14: Hello("Jason Isaacs") = "Hi, %!v(MISSING). Welcome!", <nil>, want match for `\bJason Isaacs\b`, nil
FAIL
exit status 1
FAIL    example.com/greetings   0.001s
```

| EXAMPLE: |
| :------- |
| See [05_greetings_tests](05_greetings_tests/) for a runnable example. |

## Learn how to compile and install the app

While the `go run` command is a useful shortcut for compiling and running a program while you're developing it, it doesn't generate a binary executable.

In this section we'll learn a couple of additional commands to:
+ compile the packages, along with their dependencies, without installing the results &mdash; `go build`.
+ compile and install the package &mdash; `go install`.

To compile and build the packages, type the following command in the `hello/` directory:

```bash
go build
```

After doing so, a new file `hello` would be created in the same directory. You can do:

```bash
./hello
```

to confirm it's an executable, and that it produces the same output as `go run .`.

| NOTE: |
| :---- |
| In Windows a `hello.exe` will be created. |

The `go build` command has created an executable, but it hasn't been installed. That is, to run it, you need to either be in the directory where the executable has been created, or you need to specify the path to the executable.

When you install a Go binary, you'll be able to execute without specifying its path.

The `go install` command does that by copying the binary into an specific location.

You can get that location with the command:

```bash
go list -f '{{.Target}}'
/home/ubuntu/go/bin/hello
```

You will need to run that command in a directory where a main module is found (e.g., `hello/`).

In order to be able to do:

```go
hello
```

wherever you are, you will need to either:
+ add the Go install directory to your system's path
+ change the install target by setting the `GOBIN` library

To use the first method, you just need to do:

```bash
export PATH=$PATH:/home/ubuntu/go/bin
```

Right after that, you'll be able to run the `hello` command everywhere:

```bash
$ hello
map[Florence Pugh:Great to see you Florence Pugh Idris Elba:Hi, Idris Elba. Welcome! Jason Isaacs:Hello to Jason Isaacs!]
```

For the second method, you can create a `bin/` directory in your home area, and then configure the `GOBIN` setting:

```bash
# List the current value
go env

# Set the value to some dir
go env -w GOBIN=/home/ubuntu/bin

go install

hello
map[Florence Pugh:Great to see you Florence Pugh Idris Elba:Hello to Idris Elba! Jason Isaacs:Great to see you Jason Isaacs]
```

| EXAMPLE: |
| :------- |
| See [06_greeting_build](06_greetings_build/) for a runnable example. |